# <center>6.4实战: 手写数字识别分类</center>

实验说明：本实验代码较为简单，利用Tensorflow2.x框架搭建Lenet-5模型，并使用手写体数字数据集MNIST对模型进行训练，最终能对手写体数字图片进行分类。运行该代码需要的环境如下：
Python3.5+
Tensorflow2.1.0


## 1.导入需要的python库

首先导入需要的python库，基于TensorFlow的Keras模块用于构建和训练模型。在导入Tensorflow之后，打印了其版本。以下代码均为TensorFlow2.x版本代码，若运行的版本小于2.0，则代码可能无法正常运行。

In [ ]:
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## 2.准备数据集

本实验使用的数据是MNIST数据集，数据中包含0-9十种数字的样本图片。此处如果下载数据集过慢，可以手动下载mnist.npz放置在~/.keras/datasets下。

In [2]:
mnist = tf.keras.datasets.mnist
(train_images,train_labels),(test_images,test_labels) = mnist.load_data()

## 3.构建网络结构

根据Lenet-5的结构定义逐层搭建，总计3个卷积层，2个全连接层。输出十个类别的概率值。

In [3]:
# 序贯式方法搭建LeNet网络
net = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=6,kernel_size=(5,5),activation='sigmoid',input_shape=(28,28,1)),
    tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2),
    tf.keras.layers.Conv2D(filters=16,kernel_size=(5,5),activation='sigmoid',padding="same"),
    tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2),
    tf.keras.layers.Conv2D(filters=32,kernel_size=(5,5),activation="sigmoid",padding="same"),
    #扁平化数据输入全连接层
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(200,activation='sigmoid'),
    tf.keras.layers.Dense(10,activation='sigmoid')
])

## 4.转换数据结构

将数据转换为tensorflow通用的NWHC模式。

In [4]:
# 转换数据结构,增加维度
train_images=tf.reshape(train_images,(train_images.shape[0],train_images.shape[1],train_images.shape[2],1))
print(train_images.shape)
test_images=tf.reshape(test_images,(test_images.shape[0],test_images.shape[1],test_images.shape[2],1))
print(test_images.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


## 5.模型训练与效果评估

训练、测试并保存模型。

In [5]:
#定义训练方法,超参数设置
optimizer = tf.keras.optimizers.SGD(learning_rate=0.9,momentum=0.0,nesterov=False)
net.compile(optimizer=optimizer,
           loss='sparse_categorical_crossentropy',
           metrics=['accuracy'])

# 设置训练参数,输入训练集数据,指定迭代5次,验证集按10%抽取
net.fit(train_images,train_labels,epochs=5,validation_split=0.1)
# 评估模型，得出测试的准确率
scores = net.evaluate(test_images,test_labels,verbose=1)
print(scores)
# 保存模型
net.save('model/lenet5_mnist.h5')

Train on 54000 samples, validate on 6000 samples
Epoch 1/5
54000/54000 [==============================] - 12s 213us/sample - loss: 2.3023 - accuracy: 0.1123 - val_loss: 2.3019 - val_accuracy: 0.1050
Epoch 2/5
54000/54000 [==============================] - 9s 170us/sample - loss: 0.9836 - accuracy: 0.6355 - val_loss: 0.2577 - val_accuracy: 0.9210
Epoch 3/5
54000/54000 [==============================] - 8s 156us/sample - loss: 0.1298 - accuracy: 0.9574 - val_loss: 0.0902 - val_accuracy: 0.9727
Epoch 4/5
54000/54000 [==============================] - 8s 154us/sample - loss: 0.0951 - accuracy: 0.9693 - val_loss: 0.1277 - val_accuracy: 0.9565
Epoch 5/5
10000/10000 [==============================] - 1s 72us/sample - loss: 0.0691 - accuracy: 0.9776
[0.0690730897440575, 0.9776]
